# Purpose: To build out the skeleton Pseudo Code for this project

__For now__: Moving my notes from my one one one with Elizabeth and will build out more in the next few days

1. Input .csv from experimental team to describe data
2. CSV pulls the proper images from NDP (or wherever best storage location is determined to be)
2a. There needs to be some integration with NDP to break the image into chunks with the appropriate zoom and then provide the original slice with locations of test and training images printed out.
2b. I think to do this we are going to have to register the image to some kind of atlas... or somethign to determine regions so that the code can grab pictures
3. Images need to be split
3a. Regionally
3b. Test Groups
3c. Into Small enough sizes for most efficient processing by package
3d. Magifications
4. Threshold and segment
4a. Ifthresholds (for general object identification) [July]
4b. + Skeletonization (for specific highly branched morphometric analysis) [Robin]
5. Split data into testing and training groups
6. Shape factor analysis on both models (before ML model)
7. Feed 4(1) and 4(2) separately to modified VAMPIRE package to build models
8. Store created models
9. Test models with testing data sets
10. Data output
11. Comprehensive CSV for optional additional analysis
12. Data Visualization
13. Save all data into some storage location (Google Drive?)


Other Notes: 
1. Modified VAMPIRE package: Worth forking the Wirtz lab repository now and building on by ourselves possibly – adding that fork to our repository - COMPLETE
2. Want a repository that works through Binder? Preferably so the Neonatalogy lab can do this straight through a web interface
3. Along with outputs want to output the variables used for all of the steps on days that experiments were run with an easy print out maybe for lab notebook storage? (A way to integrate the electronic lab notebook I want to get more fine tuned for our lab with a standard lab notebook - and provide some under the hood knowledge for Neonatology)

4. Add in an optimization step for what size of image is sufficient for cropping down the whole scan images 
5. Including an analysis before creating that analysis that selects the best # of shape modes and #coordinate points – maybe we could reach out the Denis Wirtz lab about this or build in our own integration



Current Goal:

1. Get a bunch of images from a slice of the ferret brain
2. Put them in a folder
3. Perform thresholding within the Jupyter Notebook
3. Integrate vampire into the Jupyter Notebook
4. Run those images and get an output within the notebook or specific folder (which should will need a results output)
5. Save all of this information to some results folder
6. See if it works in binder

Next Goals:
1. Build out ifthresholds more for immunohistochemistry stains
2. Build in NDP regional registration and automatic image breakdown
3. Integrate Google Drive
4. New visualizations based on what came from paper
5. Statistics possibly with Rthon
6. Speed up any slow processes with Cython
7. Integrate ifThresholds to pick and perform the best thresholding (or integrate a step to say whether this needs to be done or not)

*Step 1: Import necessary packages*

If import shows ERROR:

"Could not find Java JRE compatible with x86_64 architecture" then close out of jupyter lab and exit that terminal window. Start a new window and before accessing Jupyter Lab fix the Java home

http://www.sajeconsultants.com/how-to-set-java_home-on-mac-os-x/?utm_source=rss&utm_medium=rss&utm_campaign=how-to-set-java_home-on-mac-os-x

' export JAVA_HOME=/Library/Java/JavaVirtualMachines/jdk-14.0.1.jdk/Contents/Home'

' echo $JAVA_HOME'

In [157]:
import numpy as np
import os
import pandas as pd

from skimage import io
from skimage import filters
from skimage import measure
from skimage import morphology
from skimage import segmentation
from skimage import color

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

from PIL import Image

import vampireanalysis
from vampireanalysis.vampire import Model

*Step 2: User Inputs*

In [113]:
data_folder = '/Users/hhelmbre/Desktop/fiberf_data'
image_file_type = '.jpg'

*Step 3: Threshold and Segment Images*

Step 3a: Import and threshold images from 'data_folder'

In [114]:
image_list = os.listdir(data_folder)
image_list = np.asarray(image_list)

*Removing anything that isn't an image from the image list*

In [115]:
k=0
for images in image_list:
    if image_file_type in str(images):
        k+=1
    else:
        image_list = np.delete(image_list, (k), axis=0)

*back to what will be the main code*

*need to go in and add code to create a reasonable cut off for the min_size remove small objects*

*Go in and add code that does a better job at separating overlapping cells (maybe this is just in thresholding*)

In [134]:
#Creating a folder for the new tresholded images
os.mkdir(str(data_folder + '/' + 'threshold_applied_images'))

#initializing a pandas dataframe
overall_region_meas = pd.DataFrame(columns = ['Filename', 'ImageID', 'ObjectID', 'X', 'Y', 'Area', 'Perimeter', 'Major Axis', 'Minor Axis', 'Circularity', 'Aspect Ratio'])

#initializing a count
k=1

#Looping through all images in list (this task should be put to multiple CPUs in future if a large data set)
for images in image_list:
    
    
    #initializing a pandas data frame for measurement data
    region_meas = pd.DataFrame()
    
    #Going through each of the images to get their binarized images and measurement info
    name = str(data_folder + '/' + images)
    im = io.imread(name)
    im = im[:,:,2]
    threshold = filters.threshold_otsu(im)
    binary = morphology.closing(im < threshold, morphology.square(5))
    label_image = measure.label(binary, return_num=False, connectivity=2)
    binary2 = morphology.remove_small_objects(label_image, min_size=400, connectivity=2, in_place=True)
    invert_binary2 = np.invert(binary2)
    
    # Saving the thresheld images to their own folder with modified names
    im_to_save = Image.fromarray(np.uint8(binary2), mode='L')
    new_name = str(data_folder + '/' + 'threshold_applied_images' + '/' + images)
    new_name = new_name.replace('.jpg','_threshold.jpg')
    im_to_save.save(new_name)
    
    
    #Saving the region properties as a csv
    props = measure.regionprops_table(binary2, properties= ('centroid', 'area', 'perimeter', 'major_axis_length', 'minor_axis_length'))
    props_meas = pd.DataFrame(props)
    region_meas['X'] = props_meas['centroid-0']
    region_meas['Y'] = props_meas['centroid-1']
    region_meas['Area'] = props_meas['area']
    region_meas['Perimeter'] = props_meas['perimeter']
    region_meas['Major Axis'] = props_meas['major_axis_length']
    region_meas['Minor Axis'] = props_meas['minor_axis_length']
    region_meas['Circularity'] = (region_meas['Area']*4*np.pi)/(region_meas['Perimeter']**2)
    region_meas['Aspect Ratio'] = (region_meas['Major Axis']/region_meas['Minor Axis'])
    region_meas.insert(0, 'Filename', images)
    region_meas.insert(1, 'ImageID', k)
    region_meas.insert(2, 'ObjectID', np.arange(len(region_meas)))

    overall_region_meas = overall_region_meas.append(region_meas)
    k+=1

path = '/Users/hhelmbre/Desktop/fiberf_data/threshold_applied_images/'
overall_region_meas.to_csv(path+'c1_registry.csv', index=False)
                           

# Step 3: Vampire Analysis

In [ ]:
#manual create entries
#manual create a progress bar
#input the entries, buidModel? and progress_bar into def Model of vampire.py (hope that works)

In [138]:
from tkinter import *
from tkinter import filedialog
from tkinter.ttk import *

In [163]:
entries = {
    'Number of shape modes': 5,
    'Number of coordinates': 50,
    'Status': 'welcome to vampire analysis',
    'Model output folder': '/Users/hhelmbre/Desktop/fiberf_data/model',
    'Image sets to build': '/Users/hhelmbre/Desktop/fiberf_data/model/images-to-apply-model.csv',
    'Model to apply': '/Users/hhelmbre/Desktop/fiberf_data/model/images-to-apply-model.csv',
    'Model name': 'vampire_practice'
}

In [164]:
progress_bar = None

buildModel = True

In [165]:
Model(entries, buildModel, progress_bar)

AttributeError: 'str' object has no attribute 'delete'

*Step 5: Train and Test Split Images*

*Step 6: Build model using VAMPIRE*

*Step 7: Apply model using VAMPIRE*

*Step 8: New visualization of VAMPIRE data*

- This should probably be built into VAMPIRE and not just into this notebook - think about this for the future